In [1]:
import numpy as np
from scipy.optimize import minimize

from qiskit import transpile
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit, ParameterVector
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeCairo

In [2]:
backend = AerSimulator()
#backend = FakeCairo()
shots = 100
en = 2 # number of encoding layers
de = 1 # number of decoding layers
h = 4 # half size of register

# DO NOT EDIT VARIABLES BELOW

n = 2*h # full size of register (should be an EVEN number)
deviation = 1
angles = np.polynomial.hermite.hermgauss(12)[0][6:]
weights = np.polynomial.hermite.hermgauss(12)[1][6:]
eta = ParameterVector('η', 3*en)
theta = ParameterVector('θ', 3*de)
T_x = SparsePauliOp(['I'*(n-1-i)+'X'+'I'*i for i in range(n)], 0.5).power(2)
T_z = SparsePauliOp(['I'*(n-1-i)+'Z'+'I'*i for i in range(n)], 0.5).power(2)

In [3]:
def circuit(phi):
    qr = QuantumRegister(n, 'q')
    cr = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qr, cr)
    qc.ry(np.pi/2, qr)
    for i in range(en):
        qc.append(PauliEvolutionGate(T_z, time=eta[3*i]), range(n)) # T_z
        qc.append(PauliEvolutionGate(T_x, time=eta[3*i+1]), range(n)) # T_x
        qc.rx(eta[3*i+2], qr)
    qc.rz(-phi, qr)
    for i in range(de):
        qc.rx(theta[3*i], qr)
        qc.append(PauliEvolutionGate(T_x, time=theta[3*i+1]), range(n)) # T_x
        qc.append(PauliEvolutionGate(T_z, time=theta[3*i+2]), range(n)) # T_z
    qc.rx(np.pi/2, qr)
    qc.measure(qr, cr)
    return qc

def mse(phi, parameters):
    bc = circuit(phi).assign_parameters({eta: parameters[:3*en], theta: parameters[3*en:]})
    tc = transpile(bc, backend)
    result = backend.run(tc, shots=shots).result()
    counts = result.get_counts()
    data = {m: 0 for m in range(-h,h+1)}
    for outcome in counts:
        m = outcome.count('1') - h
        data[m] += counts[outcome]
    a = 0.2
    mse = 0
    for m in data:
        mse += ((phi - a*m)**2)*data[m]/shots
    return mse

def cost(parameters): 
    cost = 2*sum([mse(angles[i], parameters)*np.exp(-angles[i]**2/(2*deviation**2))/(deviation*np.sqrt(2*np.pi))*weights[i] for i in range(3)])
    return cost

In [4]:
x0 = np.random.random(3*(en+de))
costs = []
costs.append(cost(x0))
opt = minimize(cost, x0, method='Nelder-Mead', options={'disp':True}, callback=lambda x: costs.append(cost(x)))

/tmp/ipykernel_2532/1098467242.py:4: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  opt = minimize(cost, x0, method='Nelder-Mead', options={'disp':True}, callback=lambda x: costs.append(cost(x)))


In [5]:
print('Posterior width:', np.sqrt(min(costs)))

Posterior width: 0.4259103853062732
